<a href="https://colab.research.google.com/github/toanpt74/COLAB_RD/blob/main/AutoEncode_Denoise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from keras import layers
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Conv2D, BatchNormalization, ReLU, Flatten, Dense, Reshape, Conv2DTranspose,MaxPooling2D
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

def preprocess(array):
    """
    Normalizes the supplied array and reshapes it into the appropriate format.
    """
    array = array.astype("float32") / 255.0
    array = np.reshape(array, (len(array), 28, 28, 1))
    return array

def noise(array):
    """
    Adds random noise to each image in the supplied array.
    """
    noise_factor = 0.4
    noisy_array = array + noise_factor * np.random.normal(
        loc=0.0, scale=1.0, size=array.shape
    )
    return np.clip(noisy_array, 0.0, 1.0)

def display(array1, array2):
    """
    Displays ten random images from each one of the supplied arrays.
    """
    n = 10
    indices = np.random.randint(len(array1), size=n)
    images1 = array1[indices, :]
    images2 = array2[indices, :]
    plt.figure(figsize=(20, 4))
    for i, (image1, image2) in enumerate(zip(images1, images2)):
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(image1.reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(image2.reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()

(train_data, _), (test_data, _) = mnist.load_data()

# Normalize and reshape the data
train_data = preprocess(train_data)
test_data = preprocess(test_data)
# Create a copy of the data with added noise
noisy_train_data = noise(train_data)
noisy_test_data = noise(test_data)
# Display the train data and a version of it with added noise
#display(train_data, noisy_train_data)

input = layers.Input(shape=(28, 28, 1))

def encoder(encoder_inputs):
    #block 1
    x = Conv2D(32, (3, 3), activation="relu", padding="same")(encoder_inputs)
    #block 2
    x = Conv2D(64, (3, 3), activation="relu", padding="same")(x)
    y = x
    x = ReLU()(x)
    # Final Block
    flatten = layers.Flatten()(x)
    bottleneck = layers.Dense(16)(flatten)
    return bottleneck, y.shape

def decoder(x, conv_shape):
    units = conv_shape[1] * conv_shape[2] * conv_shape[3]
    x = Dense(units)(x)
    x = Reshape((conv_shape[1], conv_shape[2], conv_shape[3]))(x)
    #block 1
    x = Conv2DTranspose(64, (3,3), activation='relu', padding='same')(x)
    #block 2
    x = Conv2DTranspose(32, (3,3), activation='relu', padding='same')(x)
    decoder_outputs = Conv2DTranspose(1, 3, 1, padding='same', activation='sigmoid')(x)
    return decoder_outputs


optimizer =tf.keras.optimizers.Adam(lr=0.001)
input = layers.Input(shape=(28,28,1))
endcoder_output, conv_shape = encoder(input)
decoder_outputs = decoder(endcoder_output, conv_shape)
model = Model(input,[decoder_outputs])
model.compile(optimizer="adam", loss="binary_crossentropy")
model.summary()

model.fit(
    x=train_data,
    y=train_data,
    epochs=50,
    batch_size=128,
    shuffle=True,
    validation_data=(test_data, test_data),
)

predictions = model.predict(test_data)
display(test_data, predictions)

